In [1]:
import pandas as pd
import requests
import json
import datetime
import concurrent.futures
import swifter
import os

In [2]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [3]:
def converte_data(data_str):
    return pd.to_datetime(data_str).tz_convert('America/Sao_Paulo')


def gera_lista_assuntos(assuntos_do_df):
    lst_assuntos=[]
    for assunto in assuntos_do_df:
        try:
            lst_assuntos.append(assunto.get('nome'))
        except:
            lst_assuntos.append('')

    return lst_assuntos


def gera_lista_movimentos(movimentos):
    lst_movimentos_final =[]
    for movimento in movimentos:
        codigo = movimento.get('codigo')
        nome = movimento.get('nome')
        data_hora = movimento.get('dataHora')
        if data_hora:
            data_hora = converte_data(data_hora)
        lst_movimentos_final.append([ codigo, nome, data_hora])
    return lst_movimentos_final

def process_movimento(movimento):
    codigo = movimento.get('codigo')
    nome = movimento.get('nome')
    data_hora = movimento.get('dataHora')
    if data_hora:
        data_hora = converte_data(data_hora)
    return [codigo, nome, data_hora]

def gera_lista_movimentos_multithread(movimentos):
    lst_movimentos_final = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_movimento, movimentos))
        lst_movimentos_final.extend(results)
    return lst_movimentos_final

In [4]:
def lista_para_dataframe(dados_dict):
  processos = []
  for processo in dados_dict['hits']['hits']:
    numero_processo = processo['_source']['numeroProcesso']
    grau = processo['_source']['grau']
    classe = processo['_source']['classe']['nome']
    try:
      assuntos = processo['_source']['assuntos'] # Pode ter mais de um
    except:
      assuntos = []
    data_ajuizamento = processo['_source']['dataAjuizamento']
    ultima_atualizacao = processo['_source']['dataHoraUltimaAtualizacao']
    #formato = processo['_source']['formato']['nome']
    codigo = processo['_source']['orgaoJulgador']['codigo']
    orgao_julgador = processo['_source']['orgaoJulgador']['nome']
    municipio = processo['_source']['orgaoJulgador']['codigoMunicipioIBGE']
    sort = processo['sort'][0]
    try:
      movimentos = processo['_source']['movimentos']
    except:
      movimentos = []

    processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, \
                      codigo, orgao_julgador, municipio, grau, assuntos, movimentos, sort])

  df = pd.DataFrame(processos, columns=['numero_processo', 'classe', 'data_ajuizamento', 'ultima_atualizacao', \
                        'codigo', 'orgao_julgador', 'municipio', 'grau', 'assuntos', 'movimentos', 'sort'])
  #ddf = dd.from_pandas(df)
  #ddf.sample(frac=0.001)
  df['movimentos'] = df['movimentos'].swifter.apply(gera_lista_movimentos_multithread)
  df['assuntos'] = df['assuntos'].swifter.apply(gera_lista_assuntos)
  #df['movimentos'] = df['movimentos'].swifter.apply(gera_lista_movimentos_multithread)
  df['data_ajuizamento'] = df['data_ajuizamento'].swifter.apply(converte_data)
  df['ultima_atualizacao'] = df['ultima_atualizacao'].swifter.apply(converte_data)
  try:
    df['movimentos']= df['movimentos'].swifter.apply(lambda x: sorted(x, key=lambda tup: tup[2], reverse=False))
  except:
    pass
  #df.sample(5)
  #df.iloc[df.astype(str).drop_duplicates(inplace=True).index]
  return df

In [5]:
def criar_dataset(orgao, grau, data):
  
  df_tjrn = pd.DataFrame()
  url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjrn/_search"
  api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==" # Chave pública
  tribunal = 'TJRN'
  grau = grau
  orgaoJulgador = orgao
  size = 5000
  data = data

  payload = json.dumps(
  {
  "size": size,
  "query": {
      "bool": {
        "must": [
            {"match": {"tribunal": tribunal}},
            {"match": {"orgaoJulgador.nome": orgaoJulgador}},
            {"match": {"grau": grau}},
            {"range": {"dataAjuizamento": {"gte": data }}}
        ]
      }
  },
    "sort": [{"@timestamp": {"order": "asc"}}]
  })

  headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)  # <Response [200]>
  dados_dict = response.json() # <class 'dict'>
  df_tjrn = lista_para_dataframe(dados_dict)
  numero_processos = size

  while numero_processos == size:
    numero_processos = len(dados_dict['hits']['hits'])
    ultima_posicao_dicionario = dados_dict['hits']['hits'][(len(dados_dict['hits']['hits'])-1)]['sort'][0]
    #print(f'Partindo da posição: {ultima_posicao_dicionario}')
    payload = json.dumps(
    {
    "size": size,
    "query": {
        "bool": {
          "must": [
            {"match": {"tribunal": tribunal}},
            {"match": {"orgaoJulgador.nome": orgaoJulgador}},
            {"match": {"grau": grau}},
            {"range": {"dataAjuizamento": {"gte": data}}}
            
          ]
        }
    },
      "search_after": [ ultima_posicao_dicionario ],
      "sort": [{"@timestamp": {"order": "asc"}}]
    })

    headers = {
      'Authorization': api_key,
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)  # <Response [200]>
    dados_dict = response.json() # <class 'dict'>
    numero_processos = len(dados_dict['hits']['hits'])
    ultima_posicao_dicionario = dados_dict['hits']['hits'][(len(dados_dict['hits']['hits'])-1)]['sort']
    df_tjrn = pd.concat([df_tjrn, lista_para_dataframe(dados_dict)])
    df_tjrn = df_tjrn[df_tjrn['orgao_julgador'] == orgao]
    tamanho_dataset = len(df_tjrn.index)
    ultima_data_ajuizamento = dados_dict['hits']['hits'][len(dados_dict['hits']['hits'])-1]['_source']['dataAjuizamento']
    print(f'{datetime.datetime.now()} \tNúmero de processos: {tamanho_dataset} \t Data do último processo adicionado: {ultima_data_ajuizamento}' )
    #if tamanho_dataset > 2000000:
    #  break

  print(f'Número de processos incorporados: {tamanho_dataset}')
  
  return df_tjrn

In [ ]:
def salvar_dataset(df, orgao, grau, data):        
    orgao = orgao.replace(' ', '_')
    nome_df = 'TJRN_' + orgao + '_' + data + '.csv'
    nome_df = nome_df.replace(" ", "_")
    os.chdir('dados')
    df.to_csv(nome_df, sep=';', header=True, index=False)

In [6]:
lista_orgaos = ['1º JUIZADO ESPECIAL DA FAZENDA PÚBLICA', '2º JUIZADO ESPECIAL DA FAZENDA PÚBLICA', '3º JUIZADO ESPECIAL DA FAZENDA PÚBLICA', 
                '4º JUIZADO ESPECIAL DA FAZENDA PÚBLICA', '5º JUIZADO ESPECIAL DA FAZENDA PÚBLICA']
orgao = '4º JUIZADO ESPECIAL DA FAZENDA PÚBLICA'
grau = 'JE'
data = '2018-01-01'
for orgao in lista_orgaos:
    df = criar_dataset(orgao, grau, data)
    df.reset_index(drop=True, inplace=True)
    salvar_dataset(df, )

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/416 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/416 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/416 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/416 [00:00<?, ?it/s]

2024-05-25 04:33:46.190467 	Número de processos: 23446 	 Data do último processo adicionado: 2019-10-23T10:27:48.000Z
Número de processos incorporados: 23446


In [7]:
df.sample(2)

,numero_processo,classe,data_ajuizamento,ultima_atualizacao,codigo,orgao_julgador,municipio,grau,assuntos,movimentos,sort
8406,08786061420208205001,Cumprimento de Sentença contra a Fazenda Pública,2020-12-27 12:03:13-03:00,2022-12-11 17:49:03.720000-03:00,75496,4º JUIZADO ESPECIAL DA FAZENDA PÚBLICA,3201,JE,[],"[[26, Distribuição, 2020-12-26 21:00:00-03:00], [51, Conclusão, 2020-12-26 21:00:00-03:00], [60, Expedição de documento, 2021-01-07 21:00:00-03:00], [85, Petição, 2021-01-07 21:00:00-03:00], [246, Definitivo, 2021-02-11 21:00:00-03:00], [454, Indeferimento da petição inicial, 2021-01-06 21:00:00-03:00], [459, Ausência de pressupostos processuais, 2021-01-06 21:00:00-03:00]]",1681404219734
7502,08481804820228205001,Procedimento do Juizado Especial da Fazenda Pública,2022-07-05 12:46:31-03:00,2023-01-31 19:57:02.207000-03:00,75496,4º JUIZADO ESPECIAL DA FAZENDA PÚBLICA,3201,JE,[],"[[26, Distribuição, 2022-07-04 21:00:00-03:00], [51, Conclusão, 2022-07-04 21:00:00-03:00], [60, Expedição de documento, 2022-07-10 21:00:00-03:00], [60, Expedição de documento, 2022-07-10 21:00:00-03:00], [85, Petição, 2022-07-21 21:00:00-03:00], [85, Petição, 2022-09-14 21:00:00-03:00], [51, Conclusão, 2022-09-20 21:00:00-03:00], [60, Expedição de documento, 2022-10-30 21:00:00-03:00], [85, Petição, 2022-12-29 21:00:00-03:00], [11010, Mero expediente, 2022-07-06 21:00:00-03:00], [246, Definitivo, 2022-12-05 21:00:00-03:00], [219, Procedência, 2022-10-27 21:00:00-03:00]]",1681402684128


In [8]:
print(df['orgao_julgador'].unique())

['4º JUIZADO ESPECIAL DA FAZENDA PÚBLICA']
